In [1]:
import numpy as np
import pandas as pd
import keras
import pickle
import sklearn
import analysis_util as util
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_curve, auc, plot_roc_curve
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from keras import metrics, models, layers, optimizers
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

Using TensorFlow backend.


In [2]:
clfs = [LogisticRegressionCV(cv=5),  
        RandomForestClassifier(n_estimators=100), 
        SVC()]

In [3]:
with open("../data/X_train.txt", "rb") as fp:  
    X_train = pickle.load(fp)
    
with open("../data/smote_X_train.txt", "rb") as fp:  
    smote_X_train = pickle.load(fp)
    
with open("../data/X_train_val.txt", "rb") as fp:  
    X_train_val = pickle.load(fp)
    
with open("../data/X_test.txt", "rb") as fp:  
    X_test = pickle.load(fp)
    
    
with open("../data/y_train.txt", "rb") as fp:  
    y_train = pickle.load(fp)
    
with open("../data/smote_y_train.txt", "rb") as fp:  
    smote_y_train = pickle.load(fp)
    
with open("../data/y_train_val.txt", "rb") as fp:  
    y_train_val = pickle.load(fp)
    
with open("../data/y_test.txt", "rb") as fp:  
    y_test = pickle.load(fp)

In [5]:
neg_y_train = [1 if x == 0 else 0 for x in y_train]
neg_y_train_val = [1 if x == 0 else 0 for x in y_train_val]
neg_y_test = [1 if x == 0 else 0 for x in y_test]

In [6]:
neg_y_train_ohe = keras.utils.to_categorical(neg_y_train, 2)
neg_y_train_val_ohe = keras.utils.to_categorical(neg_y_train_val, 2)
neg_y_test_ohe = keras.utils.to_categorical(neg_y_test, 2)

In [12]:
pd.Series(neg_y_train).value_counts(normalize=True)

0    0.93782
1    0.06218
dtype: float64

In [13]:
dict(1/pd.Series(neg_y_train).value_counts(normalize=True))


{0: 1.0663030509630134, 1: 16.082262210796916}

In [15]:
class_weight = dict(1/pd.Series(neg_y_train).value_counts(normalize=True))
# class_weight = {0:1000000, 1:1}

clf_ann = models.Sequential()
clf_ann.add(layers.Dropout(.3, input_shape=(X_train.shape[1], )))
clf_ann.add(layers.Dense(1000, activation='relu'))
clf_ann.add(layers.Dropout(.3))
clf_ann.add(layers.Dense(500, activation='relu'))
clf_ann.add(layers.Dropout(.3))
clf_ann.add(layers.Dense(100, activation='relu'))
clf_ann.add(layers.Dropout(.3))
clf_ann.add(layers.Dense(1, activation='sigmoid'))
clf_ann.compile(optimizer='SGD', 
                loss='binary_crossentropy',
                metrics=[util.get_f1])
history = clf_ann.fit(X_train, neg_y_train, epochs=20, batch_size=100, class_weight=class_weight, validation_data=(X_train_val, neg_y_train_val))

Train on 6256 samples, validate on 1340 samples
Epoch 1/20
6256/6256 [==============================] - 9s 1ms/step - loss: 1.3864 - get_f1: 0.1124 - val_loss: 0.6960 - val_get_f1: 0.1192
Epoch 2/20
6256/6256 [==============================] - 7s 1ms/step - loss: 1.3863 - get_f1: 0.1177 - val_loss: 0.6907 - val_get_f1: 0.1032
Epoch 3/20
6256/6256 [==============================] - 7s 1ms/step - loss: 1.3849 - get_f1: 0.1211 - val_loss: 0.6924 - val_get_f1: 0.1156
Epoch 4/20
6256/6256 [==============================] - 7s 1ms/step - loss: 1.3847 - get_f1: 0.1251 - val_loss: 0.6861 - val_get_f1: 0.0797
Epoch 5/20
6256/6256 [==============================] - 7s 1ms/step - loss: 1.3832 - get_f1: 0.1340 - val_loss: 0.6923 - val_get_f1: 0.1370
Epoch 6/20
6256/6256 [==============================] - 7s 1ms/step - loss: 1.3814 - get_f1: 0.1289 - val_loss: 0.6866 - val_get_f1: 0.1060
Epoch 7/20
6256/6256 [==============================] - 7s 1ms/step - loss: 1.3809 - get_f1: 0.1423 - val_loss: 

In [16]:
yhat_train_val = clf_ann.predict(X_train_val)
yhat_train_val = [np.argmax(y) for y in yhat_train_val]
util.report(neg_y_train_val, yhat_train_val)

[[100.   0.]
 [100.   0.]]

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1255
           1       0.00      0.00      0.00        85

    accuracy                           0.94      1340
   macro avg       0.47      0.50      0.48      1340
weighted avg       0.88      0.94      0.91      1340



/Users/stlanier/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
for clf in clfs:
    clf.fit(X_train, neg_y_train)
    print(f'{clf}')
    print('--Train--')
    util.report(neg_y_train, clf.predict(X_train))
    print('--Validation--')
    util.report(neg_y_train_val, clf.predict(X_train_val))

/Users/stlanier/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/stlanier/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sc

LogisticRegressionCV(cv=5)
--Train--
[[100.    0. ]
 [ 14.7  85.3]]

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5867
           1       1.00      0.85      0.92       389

    accuracy                           0.99      6256
   macro avg       0.99      0.93      0.96      6256
weighted avg       0.99      0.99      0.99      6256

--Validation--
[[99.   1. ]
 [89.4 10.6]]

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      1255
           1       0.43      0.11      0.17        85

    accuracy                           0.93      1340
   macro avg       0.69      0.55      0.57      1340
weighted avg       0.91      0.93      0.92      1340

RandomForestClassifier()
--Train--
[[100.    0. ]
 [  0.3  99.7]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5867
           1       1.00      1.00      1.00       389

   

In [ ]:
## try Talos (gridsearch for neural networks)
## try ntlk.sentiment.vader for tagging pos/neg words

In [11]:
isof = IsolationForest(random_state=42).fit(X_train)

In [12]:
yhat_train = isof.predict(X_train)

In [13]:
set(yhat_train)

{1}

In [14]:
yhat_val = isof.predict(X_train_val)
set(yhat_val)

{1}

In [15]:
## not detecting any outliers (=-1) in train or val data.

In [16]:
with open("../data/bigram_X_train.txt", "rb") as fp:  
    bigram_X_train = pickle.load(fp)

In [18]:
isof_bigram = IsolationForest(random_state=42).fit(bigram_X_train)
yhat_bigram_train = isof_bigram.predict(bigram_X_train)
set(yhat_bigram_train)

{1}

In [19]:
## not detecting any outliers in train data including bigrams.